In [1]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [31]:
train = pd.read_csv("train_data.csv")
val = pd.read_csv("val_data.csv")
##internal testing
x = train.drop(columns = ["id", "price", "date", "Unique_idx"], axis = 1)
y = train["price"]
# x, x_test, y, y_test = train_test_split(x, y, test_size = .25)

In [33]:
x.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,3,1.75,2190,7021,1.0,0,2,4,7,1390,800,1953,0,98178,47.5033,-122.232,2180,7155
1,3,2.50,2370,6840,2.0,0,0,3,9,2370,0,1987,0,98119,47.6503,-122.366,1590,4400
2,2,1.00,1230,3800,1.0,0,0,3,7,1230,0,1928,0,98115,47.6797,-122.292,1610,3800
3,4,2.25,2510,9963,1.0,0,0,4,9,2200,310,1967,0,98005,47.5973,-122.177,3110,9963
4,3,1.00,1160,7491,1.0,0,0,4,6,1160,0,1917,0,98177,47.7024,-122.359,1800,2267


In [27]:
# finding best params
param_grid={'learning_rate' : [.1,.2,.3,.4,.5,.6]}
boost=AdaBoostRegressor(random_state=0,  loss="square")
regr=GridSearchCV(boost, param_grid) #Create a param grid that will try all combinations from param_grid
regr.fit(x,y)

C:\Users\ean10\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0,
                                         loss='square', n_estimators=50,
                                         random_state=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [26]:
regr.best_estimator_  #Gives parameters that minimize the lost function the best 

AdaBoostRegressor(base_estimator=None, learning_rate=0.5, loss='square',
                  n_estimators=50, random_state=0)

In this case $max\_depth=16$ and $n\_estimators=13$ is the best estimator

In [34]:
regr = AdaBoostRegressor(random_state=0, learning_rate = .5, base_estimator=DecisionTreeRegressor(max_depth=16, max_features="sqrt"),n_estimators=13, loss = "square")
regr.fit(x,y)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                       max_depth=16,
                                                       max_features='sqrt',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort=False,
                                                       random_state=None,
                                                       splitter='best'),
                  learning_rate=0.5, loss='square', n_estimators=13,
                  rando

In [35]:
feature_names= x.columns
feature_dict = { feature_names[i]: regr.feature_importances_[i] for i in range(len(feature_names))}
print("Most important feature is ", max(feature_dict, key=feature_dict.get))

Most important feature is  sqft_living


In [ ]:
# more internal testing
# y_pred = regr.predict()
# np.sqrt(mean_squared_error(y_test, y_pred))

In [36]:
ids = val["Unique_idx"]
val_x = val.drop(columns = ["id", "date", "Unique_idx"], axis = 1)
val_y = regr.predict(val_x)
d = {'price' : val_y}
output_df = pd.DataFrame(data = d, index = ids)
output_df.to_csv("Predictions.csv")